In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import c as C, h, k as k_B

%matplotlib widget

In [ ]:
# 15150-15800A, 15890-16430A, 16490-16950A
wavelens = np.linspace(15000, 17000, 2001)

chip1 = (wavelens > 15150) & (wavelens < 15800)
chip2 = (wavelens > 15890) & (wavelens < 16430)
chip3 = (wavelens > 16490) & (wavelens < 16950)
off_chip = ~(chip1 | chip2 | chip3)

ANGSTROM = 1e-10
wavelens *= ANGSTROM

N_LINES = 200
line_centers = np.random.uniform(wavelens[0], wavelens[-1], size=N_LINES)
taus = np.random.uniform(0, 3, size=N_LINES)

def planck(T, wavelens):
    return 2 * h * C**2 / wavelens**5 / (np.exp(h * C / (wavelens * k_B * T)) - 1)

star_spec = planck(6000, wavelens)
line_width = 1.5 * ANGSTROM
tau_lines = taus[:, None] * (wavelens[None, :] - line_centers[:, None])**2 / (2 * line_width**2)
abs_lines = np.product(1 - np.exp(-tau_lines), axis=0)
gain = 1e-13 * (1 + 0.1 * np.cos(2 * np.pi * wavelens / (1000 * ANGSTROM)))
noise = np.random.normal(size=wavelens.size) * 1e-6
meas_spec = -2.5 * np.log10(gain * (star_spec * abs_lines) + noise)
                        
meas_spec[off_chip] = 10
plt.figure()
plt.plot(wavelens / ANGSTROM, meas_spec)
plt.ylim(5, -1)

In [ ]:
def px0(a, b):
    return 0.1 + 0.05 * a - 0.05 * b

def px1(a, b):
    return -0.5 - 0.1 * a - 0.11 * b

def px2(a, b):
    return 0.9 * (a - 2)**2 - 0.3 * (b - 6)**2

a0, b0 = 2, 6
mini_spec = np.array([px0(a0, b0), px1(a0, b0), px2(a0, b0)])

plt.figure()
_ = plt.plot(mini_spec)

In [ ]:
a, b = np.linspace(1, 3, 10), np.linspace(5, 10, 20)
a, b = np.meshgrid(a, b)
mini_spec = np.array([px0(a, b), px1(a, b), px2(a, b)])

plt.figure()
_ = plt.plot(mini_spec.reshape(3, -1), '.')

In [ ]:
fig, axes = plt.subplots(subplot_kw={'projection':'3d'})
surf = axes.plot_surface(a, b, mini_spec[2], rstride=1, cstride=1, cmap='seismic',
                       linewidth=0, antialiased=False)